<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DA/01%20HW/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# Всё необходимое для выполнения данной лабораторной работы!
import requests
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
pd.set_option('display.max_columns', 100)

## Задание 1. Парсинг - 10 баллов

В этом задании Вам предстоит работать с API сайта <a target="_blank" href="hh.ru">hh.ru</a>. Документация доступна по <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">ссылке</a>.

**Задача:**
1. выбрать 5 интересующих вас профессий (не смежных, т.е. Аналатик и Data Engineer не считается корректным)
2. затем получить доступные по данным профессиям вакансии в Москве и выделить их физические координаты
3. нанести полученные координаты на карту местоположения компаний

*Пример запроса для профессии:*

In [46]:
URL = 'https://api.hh.ru/vacancies'

params = {
    'text': "Физик-ядерщик",
    'area': 1,
    'page': 0,
    'per_page': 10
}

data = requests.get(URL, params).json()
data

{'items': [],
 'found': 0,
 'pages': 1,
 'page': 0,
 'per_page': 10,
 'clusters': None,
 'arguments': None,
 'fixes': None,
 'suggests': None,
 'alternate_url': 'https://hh.ru/search/vacancy?area=1&enable_snippets=true&items_on_page=10&text=%D0%A4%D0%B8%D0%B7%D0%B8%D0%BA-%D1%8F%D0%B4%D0%B5%D1%80%D1%89%D0%B8%D0%BA'}

Если Ваш запрос прошел успешно, то вы получите следующие ключи (значения данных ключей вы найдете по ссылке на документацию):

In [47]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

Выберете для нас 5 наиболее интересных специальностей и получите по ним имеющиеся вакансии:

In [48]:
URL_areas = 'https://api.hh.ru/areas'
URL_professions = 'https://api.hh.ru/professional_roles'
URL_vacancies = 'https://api.hh.ru/vacancies'

In [49]:
data_areas = requests.get(URL_areas).json()
sleep(0.5)
data_professions = requests.get(URL_professions).json()

In [50]:
def get_city_id_by_name(data:dict=data_areas, county_name:str='Россия', city_name:str='Москва') -> str | None:
    """
    Returns the city ID given a country name and city name.

    :param data: A dictionary containing country and city data.
    :param county_name: The name of the country.
    :param city_name: The name of the city.
    :return: The ID of the city if found, otherwise None
    """
    for country in data_areas:
        if country.get('name', []) == county_name:
            for city in country['areas']:
                if city.get('name', []) == city_name:
                    return city.get('id', None)

In [51]:
def get_random_professions_names(data:dict=data_professions, n:int=5) -> list:
    """
    Returns a list of random profession names.

    :param data: A dictionary containing professions data.
    :param n: The number of profession names to return.
    :return: A list of random profession names
    """
    result = []
    profession_categories = [profession_categories.get('name') for profession_categories in data.get('categories')][:-1] #Категорию "другое" не рассматриваем
    # Выберем по 1 случайной профессии из n случайных категорий
    for profession_category in np.random.choice(profession_categories, size=n, replace=False):
        for profession in data_professions.get('categories'):
            if profession.get('name', []) == profession_category:
                result.append(np.random.choice(profession.get('roles'), size=1, replace=False)[0].get('name'))
                break
    return result


In [52]:
def get_vacancies_page(vacancy_name:str, page:int=0, city_id:str='1') -> dict:
    """
    Fetches a page of vacancies based on the vacancy name and city.

    :param vacancy_name: The name of the vacancy to search for.
    :param page: The page number to fetch (defaults to 0).
    :param city: The city to filter vacancies by (defaults to 'Москва').
    :return: A JSON object containing the vacancies or None if an error occurs.
    """
    params = {
        'text': vacancy_name,
        'area': city_id,
        'page': page,
        'per_page': 100
    }
    try:
        response = requests.get(URL_vacancies, params)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching vacancies: {e}")
        return None
    return response.json()

In [53]:
def get_vacancies(vacancy_name, city='Москва'):
    """
    Retrieves all vacancies matching the specified vacancy name and city by paginating through the API results.

    :param vacancy_name: The name of the vacancy to search for.
    :param city: The city to filter the vacancies by (defaults to 'Москва').
    :return: A dictionary containing a list of vacancies and the total number found.
    """
    city_id = get_city_id_by_name(city_name=city)
    page = 0
    vacancies = []
    current_page = get_vacancies_page(vacancy_name=vacancy_name, page=page, city_id=city_id)
    if not current_page:
        return{'items': [], 'found': 0, 'fetched': 0}

    vacancies.extend(current_page.get('items', []))
    total_pages = current_page.get('pages', 0)
    total_vacancies = current_page.get('found', 0)

    for page in range(1, total_pages):

        sleep(np.random.rand() + 0.5)

        current_page = get_vacancies_page(vacancy_name=vacancy_name, page=page, city_id=city_id)
        if not current_page:
            break
        vacancies.extend(current_page.get('items', []))

    return {'items': vacancies, 'found': total_vacancies, 'fetched': len(vacancies)}

In [59]:
np.random.seed(42)
vacancies_info = dict()
data = []
for vacancy_name in get_random_professions_names(n=5):
    vacancies = get_vacancies(vacancy_name=vacancy_name)
    data.extend(vacancies.get('items'))
    vacancies_info[vacancy_name] = {
        'found': vacancies.get('found'),
        'fetched': vacancies.get('fetched')
    }
    print(f'{vacancies.get("fetched")} of {vacancies.get("found")} vacancies for {vacancy_name} fetched')

9 of 9 vacancies for Артист, актер, аниматор fetched
53 of 53 vacancies for Агроном fetched
116 of 116 vacancies for Автомойщик fetched
555 of 555 vacancies for Финансовый директор (CFO) fetched
2000 of 7691 vacancies for Психолог fetched


In [63]:
data[:1]

[{'id': '107241661',
  'premium': False,
  'name': 'Актриса-актер (Снегурочка)',
  'department': None,
  'has_test': False,
  'response_letter_required': False,
  'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
  'salary': {'from': 100000, 'to': None, 'currency': 'RUR', 'gross': False},
  'type': {'id': 'open', 'name': 'Открытая'},
  'address': None,
  'response_url': None,
  'sort_point_distance': None,
  'published_at': '2024-09-13T11:20:14+0300',
  'created_at': '2024-09-13T11:20:14+0300',
  'archived': False,
  'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=107241661',
  'show_logo_in_search': None,
  'insider_interview': None,
  'url': 'https://api.hh.ru/vacancies/107241661?host=hh.ru',
  'alternate_url': 'https://hh.ru/vacancy/107241661',
  'relations': [],
  'employer': {'id': '1376090',
   'name': 'Море Шоу',
   'url': 'https://api.hh.ru/employers/1376090',
   'alternate_url': 'https://hh.ru/employer/1376090',
   'logo_urls

Посчитайте сколько всего найдено вакансий:

In [60]:
vacancies_info

{'Артист, актер, аниматор': {'found': 9, 'fetched': 9},
 'Агроном': {'found': 53, 'fetched': 53},
 'Автомойщик': {'found': 116, 'fetched': 116},
 'Финансовый директор (CFO)': {'found': 555, 'fetched': 555},
 'Психолог': {'found': 7691, 'fetched': 2000}}

In [61]:
print(f" Всего вакансий найдено: {sum(v['found'] for v in vacancies_info.values())}")


 Всего вакансий найдено: 8424


Создайте структуру данных, которая будет хранить в себе имя компании и ее координаты (широта и долгота):

In [55]:
from collections import namedtuple

Company = namedtuple('Company', ['name', 'lat', 'lon'])

Создайте экземпляр этой структуры данных для каждой компании, которая выставила вакансию:

Найдите количество вакансий, у которых заданы координаты (значения данных может быть пустым):

Отберите вакансии, которые расположены в пределах МКАД (можно грубо):

Постройте график в координатах "широта-долгота" для отображения вакансий внутри МКАД (не забывайте прописать прозрачность, чтобы увидеть скопления):

**Сделайте выводы о проделанной работе:**

Вам предстоит проанализировать датасет, содержащий информацию об бриллианта, их стоимости и остальных характеристиках.



Поясним значения хранящиеся в колонках:
*   `cut` - огранка бриллианта (относится к одной из примерно 10 наиболее распространенных)
*   `color` - цвет бриллианта, прозрачные бриллианты имеют градацию `D-Z`, "более высокие" буквы - более желтоватые, но часто имеют лучшие значения, поскольку цвет трудно определить один раз в кольце
*   `clarity` - чистота бриллианта, наличие включений (внутренних дефектов), чем меньше и мельче, тем лучше
*   `carat_weight` - масса бриллианта (слабо связана с размером бриллианта, т.к. огранка и качество огранки, как правило, играют большую роль)
*   `cut_quality` - качество ограники по системе оценки GIA Cut
*   `lab` - лаборатория оценивания
*   `meas_length` - длина бриллианта
*   `meas_width` - ширина бриллианта
*   `meas_depth` - глубина бриллианта
*   `total_sales_price` - цена бриллианта в долларах.
*    **и прочие колонки**

Скачаем данные:

In [56]:
!gdown 10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8
# если работайте в jupyter то скачайте датасет с https://drive.google.com/uc?id=10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8

Downloading...
From: https://drive.google.com/uc?id=10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8
To: /content/diamonds.csv
100% 35.4M/35.4M [00:00<00:00, 65.4MB/s]


Создадим DataFrame:

In [57]:
import pandas as pd

diamonds = pd.read_csv("diamonds.csv")

Выведете первые 5 строк датасета:

Найдите количество бриллиантов всего в данном датасете:

Выведите максимальную и минимальную цену бриллианта в формате `номер - цена`:

Посчитайте количество пропусков (пропуском считается значение `unknown` или `None`):

Найдите в каких столбцах присутствуют пропуски:

В нашем датасете присутствуют колонки, которые начинаются с `fancy_color_` или `fluor_`. Большинство значений в этих колонках - `unknown` или `None`, поэтому мы считаем их малоинформативными.

Ваша задача:

найти количество таких колонок, которые начинаются с `fancy_color_` или `fluor_`:

найти количество информативных (не `None` или `unknown`) значений в этих колонках:

удалить данные колонки из датасета, сохранив изменения (в дальнейшем работаем измененным датасетом):

Проделайте тоже самое для колонок, начинающихся с `culet_`:

Посчитайте сколько бы стоил (в тысячах долларов) сейф, в котором хранились бы все бриллианты из датасета:

Немного необоснованных манипуляций.

Выберите из датасета строки с четными индексами и выведете значения (не меняя нумерацию индексов):

А теперь с индексами, кратными 6:

Выберете из датасета только категорильные признаки и выведете датасет:

### Построение графиков

**Требования к графику:**

1.   График должен быть подписан
2.   Оси должны быть подписаны
3.   Должна быть легенда графика
4.   График должен быть информативным (желательно без сильных выбросов)

**Все надписи на русском языке!**

>**Примечание**
>
>График необходимо строить для очищенных данных - в них не должно быть пропусков по осям, по которым строите график. При этом **не перезатирайте** имеющийся датафрейм!

Постройте график `цвет бриллианта - стоимость`:

Постройте график `лабараторная оценка - кол-во бриллиантов`:

Постройте график `вес бриллианта - стоимость`:

Постройте график `система оценки GIA Cut - кол-во бриллиантов`:

Сделайте выводы по построенным графикам